# preprocessing

> Collections of functions from `adata` simplifiying preprocessing

In [ ]:
#| default_exp preprocessing

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
# NOTE: needed for python 3.10 forward compatibility with scanpy as 
# scanpy uses Iterable which is deprecated in 3.10
import collections.abc
#hyper needs the four following aliases to be done manually.
collections.Iterable = collections.abc.Iterable
collections.Mapping = collections.abc.Mapping
collections.MutableSet = collections.abc.MutableSet
collections.MutableMapping = collections.abc.MutableMapping

In [ ]:
#| export
import copy, warnings
from typing import Sequence, Tuple
import scanpy as sc

In [ ]:
#| export
from degex.static import (
    GENE_IDS, TOTAL_COUNTS, 
    PCT_COUNTS_MITO, PCT_COUNTS_RIBO, DOUBLET_SCORES,
)
from degex.types import (
    AnnData, Graph, CutoffSpec, CutoffSpecs, 
)

from degex.adata import (
    set_gene_symbol_as_var_names,
    set_var_names_as_gene_ids,
    score_doublets, apply_filter_by_cutoffs,
    remove_mitochondrial_genes,
    add_gene_detection_layer,
    add_prenormalization_layer,
    sqrt_library_size_normalize,
    add_batch_mean_center_layer,
    score_genes_cell_cycle_with_batch_mean_center_data,
    select_hvg_per_batch,
    run_pca, run_pca_on_hvg,
    run_phate_on_hvg, run_phate_using_g, run_magic,
)

from degex.adata import (
    PCA_KWARGS, PHATE_KWARGS, G_KWARGS, MAGIC_KWARGS
)

In [ ]:
#| export
def prepare_h5ad_file(filename: str, plot: bool = False) -> AnnData:
    try:
        adata = sc.read_10x_h5(filename, gex_only = True)
    except ValueError:
        warnings.warn('Failed to use sc.read_10x_h5 to load file. Using read_h5ad as fallback. Is your data from 10x?')
        adata = sc.read_h5ad(filename)
    adata = set_gene_symbol_as_var_names(adata)
    try:
        adata = set_var_names_as_gene_ids(adata)
    except KeyError:
        warnings.warn(f'Failed to find a feature named {GENE_IDS} in adata.var. Not setting adata.names to `{GENE_IDS}`.')
    adata = score_doublets(adata, plot)
    return adata

def filter_pipeline(
    adata: AnnData,
    cutoff_specs: CutoffSpecs = [
        CutoffSpec(TOTAL_COUNTS,     500,  10000),
        CutoffSpec(PCT_COUNTS_MITO, None, 15),
        CutoffSpec(PCT_COUNTS_RIBO, None, 15),
        CutoffSpec(DOUBLET_SCORES,  None, 0.4),
    ],
    min_cells: int = 5,
    min_genes: int = 200,
    remove_mt_genes: bool = False,
) -> AnnData:    
    if cutoff_specs is not None:
        adata = apply_filter_by_cutoffs(adata, cutoff_specs)
    
    if min_genes:
        sc.pp.filter_cells(adata, min_genes=min_genes)
    
    if min_cells:
        sc.pp.filter_genes(adata, min_cells=min_cells)
    
    if remove_mt_genes:
        adata = remove_mitochondrial_genes(adata)    
    return adata

def normalization_pipeline(
    adata:AnnData,
    s_genes: Sequence[str] = None,
    g2m_genes: Sequence[str] = None
) -> AnnData:
    adata = add_prenormalization_layer(adata)
    adata = add_gene_detection_layer(adata)
    adata = sqrt_library_size_normalize(adata)
    adata = add_batch_mean_center_layer(adata)
    if s_genes is not None and g2m_genes is not None:
        adata = score_genes_cell_cycle_with_batch_mean_center_data(
            adata, s_genes, g2m_genes
        )
    return adata

def embedding_pipeline(
    adata: AnnData,

    # PCA on adata.X
    pca_kwargs: dict = PCA_KWARGS,
    plot_scree: bool = False,
    
    # PHATE on pca
    phate_kwargs = PHATE_KWARGS,
    g_kwargs = G_KWARGS,
    
    do_hvg: bool = True,

    # How to calc hvg
    hvg_kwargs: dict = dict(cutoff=None, percentile=90),

    # PCA on hvg
    hvg_pca_kwargs: dict=None,
    
    # PHATE on hvg
    hvg_phate_kwargs: dict = None,
    hvg_g_kwargs: dict = None,
    
    # MAGIC on g_hvg
    do_magic: bool = True,
    magic_kwargs: dict = MAGIC_KWARGS
) -> Tuple[AnnData, Graph, Graph]:
    g, g_hvg = None, None

    # STEP 1: PCA on adata.X --> X_pca
    adata = run_pca(adata, pca_kwargs=pca_kwargs, plot_scree=plot_scree)
    
    # STEP 2: PHATE on X_pca --> X_phate
    adata, g = run_phate_using_g(adata, g, phate_kwargs, g_kwargs)
    
    if do_hvg:
        # STEP 3: setup HVG params
        if hvg_pca_kwargs is None:
            hvg_pca_kwargs = copy.deepcopy(pca_kwargs)

        if hvg_phate_kwargs is None:
            hvg_phate_kwargs = copy.deepcopy(phate_kwargs)

        if hvg_g_kwargs is None:
            hvg_g_kwargs = copy.deepcopy(g_kwargs)

        # STEP 4: calculate HVGs per batch
        adata = select_hvg_per_batch(adata, hvg_kwargs)

        # STEP 5: PCA on adata.X[:, HVGs] --> X_pca_hvg
        adata = run_pca_on_hvg(adata, hvg_pca_kwargs, plot_scree)

        # STEP 6: PHATE on X_pca_hvg --> X_phate_hvg
        adata, g_hvg = run_phate_on_hvg(adata, g_hvg, hvg_phate_kwargs, hvg_g_kwargs)

        # STEP 7: MAGIC on g_hvg --> X_magic
        if do_magic:
            adata = run_magic(adata, g_hvg, magic_kwargs)

    return adata, g, g_hvg

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()